# Early Interactive Demo of Satelite Similarity Search
To run this notebook, you must download the data from [sat-searcher on Google Drive](https://drive.google.com/drive/folders/1lac_YcJHp_6GlVFZo4AE3wN4qhx3mrct?usp=drive_link). Unzip the centroids and embeddings_8bit folders and place in the outputs directory.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw

import tools

In [ ]:
centroids, embeddings = tools.load_embeddings(
    centroid_dir='./outputs/centroids/', 
    embedding_dir='./outputs/embeddings_8bit/')
index = tools.index_embeddings(embeddings)

In [ ]:
boundaries = []
for f in ['israel', 'bali', 'alabama']:
    boundaries.append(gpd.read_file(f'./data/boundaries/{f}.geojson'))

center = [32.3182, -86.9023] # center on alabama
m = ipyl.Map(basemap=ipyl.basemaps.Esri.WorldImagery, center=center, zoom=8, scroll_wheel_zoom=True)
# set the map size
m.layout.height = '800px'

# add the boundaries to the map with a thin white line and transparent fill
for b in boundaries:
    m.add_layer(ipyl.GeoData(geo_dataframe=b,
                             style={'color': 'white', 'fillOpacity': 0.0}))
# create an empty layer that I can use to write geometries to on click
search_data = ipyl.GeoJSON(data={'type': 'FeatureCollection', 'features': []}, style={'color': 'blue'})
result_data = ipyl.GeoJSON(data={'type': 'FeatureCollection', 'features': []}, style={'color': 'red'})
result_data_8_bit = ipyl.GeoJSON(data={'type': 'FeatureCollection', 'features': []}, style={'color': 'yellow'})

m.add_layer(search_data)
m.add_layer(result_data)
m.add_layer(result_data_8_bit)

# create two buttons on the map. One that is called m.pos and one that is called m.neg
# m.pos should be green and have a checkmark, m.neg should be red and have an x

m.pos = ipyw.Button(description='✅', button_style='success')
m.neg = ipyw.Button(description='❌', button_style='danger')
m.pos.on_click(lambda b: setattr(m, 'mode', 'pos'))
m.neg.on_click(lambda b: setattr(m, 'mode', 'neg'))
m.pos.layout.width = '35px'
m.neg.layout.width = '35px'
m.pos.layout.margin = '3px'
m.neg.layout.margin = '3px'

# put them on the map
mode_selector = ipyl.WidgetControl(widget=ipyw.VBox([m.pos, m.neg]),
                                   position='bottomright')
m.add_control(mode_selector)
m.mode = 'pos'


m.pos_vectors = []
m.neg_vectors = []

# get coordinates when clicking on the map
def click(**kwargs):
    if kwargs.get('type') == 'click':
        # clear the result data
        #search_data.data = {'type': 'FeatureCollection', 'features': []}
        click_y, click_x = kwargs.get('coordinates')
        tile_index, _ = tools.get_neighbors(np.array([click_x, click_y]),
                                      centroids,
                                      metric='euclid',
                                      n=1)
        tile_index = tile_index[0]
        tile_geom = tools.tile_from_point(centroids[tile_index][0],
                                    centroids[tile_index][1])
        # add the layer in blue to result data
        search_data.data = tile_geom
        tile_embedding = embeddings[tile_index]
        if m.mode == 'pos':
            m.pos_vectors.append(tile_embedding)
        elif m.mode == 'neg':
            m.neg_vectors.append(tile_embedding)
        
        if len(m.neg_vectors) == 0:
            search_embedding = np.mean(m.pos_vectors, axis=0)
        else:
            search_embedding = 2 * np.mean(m.pos_vectors, axis=0) - np.mean(m.neg_vectors, axis=0)
        tools.retrieve_neighbors(search_embedding,
                           index,
                           result_data,
                           centroids,
                           threshold=200000,
                           n=200)
m.on_interaction(click)
m